In [1]:
from docxtpl import * 
from typing import Dict

ModuleNotFoundError: No module named 'docxtpl'

# Site references

In [ ]:
#cspell:disable
PROJECT_INFO:Dict[str,any] = {
    'name':'Proyecto Fotovoltaico CESFAM San Pejerrey',
    'type':{
        'tech':'Fotovoltaico',
        'size':100,
        'unit':'kWp',
        'surface':1000
        },
    'building':{
        'name':'CESFAM El Pejerrey',
        'address':'Calle Falsa 124',
        'city':'Quillota'
    },
    'geolocation':{
        'latitude':-33.0,
        'longitude':-70.0,
        'altitude':0,
    },
}

